In [ ]:
%matplotlib widget
from pyhots.POKERDVS import POKERDVS
from spike_data_augmentation.datasets.dataloader import Dataloader
import spike_data_augmentation.transforms as transforms
import spike_data_augmentation.representations as representations
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import ipdb
import numpy as np

In [ ]:
%%capture
transform = transforms.Compose([transforms.DropEvent(drop_probability=0.0)])
representation = representations.Timesurface(surface_dimensions=(11,11), tau=5e3, merge_polarities=True)

testset = POKERDVS(file_dir='/home/gregorlenz/Development/Github/HOTS-DOJO/Datasets/Cards/usable/pips',
                   transform=transform, representation=representation)

### read all timesurfaces and associated labels from the first layer

In [ ]:
testloader = Dataloader(testset, shuffle=True)
testiterator = iter(testloader)

all_surfaces = []
all_labels = []
for surfaces, label in testiterator:
    all_surfaces.append(surfaces)
    all_labels.append(label)
    counts[label] += 1
    print('Processed', end='')
    for key, value in counts.items():
        print(' ' + str(value) + ' ' + key + ',', end='')
    print('.')


In [ ]:
all_surfaces.flatten()

### Cluster all timesurfaces

In [ ]:
n_of_centers = 16
kmeans = KMeans(n_clusters=n_of_centers).fit(surfs)
centers = kmeans.cluster_centers_.reshape(-1,11,11)

fig, axes = plt.subplots(4, 4)
axes = np.reshape(axes, -1)

for index, axis in enumerate(axes):
    axis.imshow(centers[index], vmin=0, vmax=1,
                cmap = plt.cm.hot, interpolation = 'none', origin = 'upper')

### Cut the array of timesurfaces and labels along recording borders and make histograms for each recording

In [ ]:
print('Original labels for events')
print(labels[2730:2740])
print(labels.shape)
print('Kmeans labels')
print(kmeans.labels_[2730:2740])
print(kmeans.labels_.shape)



kmeans_split = np.split(kmeans.labels_, recording_indices)
label_split = np.split(labels, recording_indices)

hists = []
[hists.append(np.histogram(x, bins=np.arange(0,n_of_centers+1))[0]) for x in kmeans_split]
short_labels = []
[short_labels.append(x[0]) for x in label_split]

print(short_labels)

import collections
collections.Counter(short_labels)

In [ ]:
# split X and y into training and testing sets
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
# fit the model with data
logreg.fit(X_train,y_train)
y_pred=logreg.predict(X_test)

In [ ]:
all_surfaces[3].shape